In [57]:
import MySQLdb
import pandas as pd

In [3]:
# used to find out which python jupyter uses.
!which python

/Users/chadleonard/miniconda2/envs/tutorial/bin/python


In [156]:
db = MySQLdb.connect(
                 host="localhost", 
                 user="root", 
                 passwd="mysqlpassword@2018", 
                 db="country_club" 
               )
cursor = db.cursor()
cols_sql = "SELECT COLUMN_NAME FROM INFORMATION_SCHEMA.COLUMNS WHERE table_name = 'facilities' \
      order by ordinal_position ";
cursor.execute(cols_sql)
cols_fac = [col[0] for col in cursor.fetchall()]
print cols_fac

cols_sql = "SELECT COLUMN_NAME FROM INFORMATION_SCHEMA.COLUMNS WHERE table_name = 'bookings' \
      order by ordinal_position ";
cursor.execute(cols_sql)
cols_book = [col[0] for col in cursor.fetchall()]
print cols_book

cols_sql = "SELECT COLUMN_NAME FROM INFORMATION_SCHEMA.COLUMNS WHERE table_name = 'members' \
      order by ordinal_position ";
cursor.execute(cols_sql)
cols_mem = [col[0] for col in cursor.fetchall()]
print cols_mem

['facid', 'name', 'membercost', 'guestcost', 'initialoutlay', 'monthlymaintenance']
['bookid', 'facid', 'memid', 'starttime', 'slots']
['memid', 'surname', 'firstname', 'address', 'zipcode', 'telephone', 'recommendedby', 'joindate']


Q1: Some of the facilities charge a fee to members, but some do not.
Please list the names of the facilities that do.

In [148]:
sql = "select * from facilities where membercost > 0.0";

cursor.execute(sql)
df_fac = pd.DataFrame([col for col in cursor.fetchall()], columns=cols_fac)
print df_fac['name'].values


['Tennis Court 1' 'Tennis Court 2' 'Massage Room 1' 'Massage Room 2'
 'Squash Court']


Q2: How many facilities do not charge a fee to members? 

In [146]:
#cursor = db.cursor()
sql = "select count(*) from facilities where membercost = 0.0";

cursor.execute(sql)
df_fac = pd.DataFrame([col for col in cursor.fetchall()], columns=['col_count'])
df_fac['col_count'].values[0]


4

Q3: How can you produce a list of facilities that charge a fee to members,
where the fee is less than 20% of the facility's monthly maintenance cost?
Return the facid, facility name, member cost, and monthly maintenance of the
facilities in question.

In [176]:
cols = ['facid', 'name', 'membercost', 'monthlymaintenance', 'm' ]
sql = "select facid, name, cast(membercost as decimal(4,2)) membercost, monthlymaintenance, \
            (.20 * monthlymaintenance ) m from facilities \
       where cast(membercost as decimal(4,2)) < (.20 * monthlymaintenance)";

cursor.execute(sql)
#cursor.fetchall()
df_fac = pd.DataFrame([col for col in cursor.fetchall()], columns=cols)
print df_fac[cols[:4]]


   facid             name membercost  monthlymaintenance
0      0   Tennis Court 1       5.00                 200
1      1   Tennis Court 2       5.00                 200
2      2  Badminton Court       0.00                  50
3      3     Table Tennis       0.00                  10
4      4   Massage Room 1       9.90                3000
5      5   Massage Room 2       9.90                3000
6      6     Squash Court       3.50                  80
7      7    Snooker Table       0.00                  15
8      8       Pool Table       0.00                  15


In [69]:
# booking

ret_sql = "select * from bookings";

cursor.execute(cols_sql)

cols = [col[0] for col in cursor.fetchall()]

print cols

cursor.execute(ret_sql)
df_bookings = pd.DataFrame([col for col in cursor.fetchall()], columns=cols)
db.close()
df_bookings.head()

['bookid', 'facid', 'memid', 'starttime', 'slots']


,bookid,facid,memid,starttime,slots
0,0,3,1,2012-07-03 11:00:00,2
1,1,4,1,2012-07-03 08:00:00,2
2,2,6,0,2012-07-03 18:00:00,2
3,3,7,1,2012-07-03 19:00:00,2
4,4,8,1,2012-07-03 10:00:00,1


['bookid', 'facid', 'memid', 'starttime', 'slots']


,bookid,facid,memid,starttime,slots
0,0,3,1,2012-07-03 11:00:00,2
1,1,4,1,2012-07-03 08:00:00,2
2,2,6,0,2012-07-03 18:00:00,2
3,3,7,1,2012-07-03 19:00:00,2
4,4,8,1,2012-07-03 10:00:00,1


In [68]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4043 entries, 0 to 4042
Data columns (total 5 columns):
bookid       4043 non-null int64
facid        4043 non-null int64
memid        4043 non-null int64
starttime    4043 non-null object
slots        4043 non-null int64
dtypes: int64(4), object(1)
memory usage: 189.5+ KB
